In [ ]:
from flask import Flask, render_template, request
from keras.preprocessing import image
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.models as models
from torchvision import transforms

import cv2
import time
import random
from PIL import Image

app = Flask(__name__)

# Loading pretrained model
base_model = models.resnet18(pretrained = True)
base_model.eval()

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

image_transform = transforms.Compose([        # Defining a variable transforms
    transforms.Resize(256),                # Resize the image to 256×256 pixels
    transforms.CenterCrop(224),            # Crop the image to 224×224 pixels about the center
    transforms.ToTensor(),                 # Convert the image to PyTorch Tensor data type
    transforms.Normalize(                  # Normalize the image
    mean=[0.485, 0.456, 0.406],            # Mean and std of image as also used when training the network
    std=[0.229, 0.224, 0.225]      
    )])

def predict_label(img_path):
    image = Image.open(img_path)
    image.resize((IMAGE_WIDTH,IMAGE_HEIGHT))
    img = image_transform(image)
    batch_t = torch.unsqueeze(img, 0)
    out = base_model(batch_t)
    with open('imagenet_classes.txt') as f:
      classes = [line.strip() for line in f.readlines()]
    _, indices = torch.sort(out, descending=True)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    ans = [(classes[idx], percentage[idx].item()) for idx in indices[0][:4]]
    print(ans)
    return ans[0][0]

def take_picture():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("test")
    img_counter = random.randrange(0, 10000, 3)
    take_one = 0;
    while True:
        ret, frame = cam.read()
        if not ret:
            print("failed to grab frame")
            break
        cv2.imshow("test", frame)
        
        time.sleep(2)
        img_name = "static/opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
        take_one += 1

        if take_one ==1:
            break

    cam.release()
    cv2.destroyAllWindows()
    
    return img_name


@app.route("/", methods=['GET', 'POST'])
def main():
    return render_template("index.html")

@app.route("/about")
def about_page():
    return "Please subscribe  Artificial Intelligence Hub..!!!"

@app.route("/submit", methods = ['GET', 'POST'])
def get_output():
    if request.method == 'POST':
        img = request.files['my_image']

        img_path = "static/" + img.filename	
        img.save(img_path)

        p = predict_label(img_path)

    return render_template("index.html", prediction = p, img_path = img_path)

@app.route("/click_image",methods = ['GET','POST'])
def take_pic():
    if request.method == 'POST':
        img = take_picture()
        p = predict_label(img)
    print(p)
    return render_template("index.html", prediction = p, img_path = img)

if __name__ =='__main__':
    #app.debug = True
    app.run(debug = True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jan/2022 14:22:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2022 14:22:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Jan/2022 14:23:15] "POST /submit HTTP/1.1" 200 -


[('Egyptian cat', 41.35151672363281), ('lynx, catamount', 15.576092720031738), ('tabby, tabby cat', 14.351587295532227), ('tiger cat', 10.5465726852417)]
static/opencv_frame_9840.png written!


127.0.0.1 - - [08/Jan/2022 14:23:41] "POST /click_image HTTP/1.1" 200 -


[('tench, Tinca tinca', 34.6447868347168), ('coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch', 33.70085525512695), ('shower cap', 1.715738296508789), ('gar, garfish, garpike, billfish, Lepisosteus osseus', 1.3448104858398438)]
tench, Tinca tinca


127.0.0.1 - - [08/Jan/2022 14:23:41] "GET /static/opencv_frame_9840.png HTTP/1.1" 200 -


static/opencv_frame_9936.png written!


127.0.0.1 - - [08/Jan/2022 14:24:30] "POST /click_image HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2022 14:24:30] "GET /static/opencv_frame_9936.png HTTP/1.1" 200 -


[('zebra', 87.37303924560547), ('chambered nautilus, pearly nautilus, nautilus', 9.150196075439453), ('lionfish', 2.842193841934204), ('tiger, Panthera tigris', 0.10806537419557571)]
zebra
